In [1]:
import tensorflow as tf
import os

In [2]:
import cv2
import imghdr

In [3]:
# data_dir = 'data' 
data_dir = 'test' 

In [4]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, image_size=(64,64))

Found 952 files belonging to 5 classes.


In [5]:
data_iterator = data.as_numpy_iterator()

In [6]:
batch = data_iterator.next()

2023-04-06 18:06:10.641316: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [7]:
data = data.map(lambda x,y: (x/255, y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [9]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [11]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu", input_shape=(64, 64, 3), kernel_initializer='he_uniform'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(23, activation="softmax", kernel_initializer='glorot_uniform')
])

In [12]:
model2.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [13]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 512)       1180160   
                                                                 
 flatten (Flatten)           (None, 73728)             0

In [14]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [16]:
hist = model2.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
21/21 [==============================] - 10s 449ms/step - loss: 1.7304 - accuracy: 0.2932 - val_loss: 1.4398 - val_accuracy: 0.4323
Epoch 2/20
21/21 [==============================] - 10s 464ms/step - loss: 1.6591 - accuracy: 0.3289 - val_loss: 1.4624 - val_accuracy: 0.3854
Epoch 3/20
21/21 [==============================] - 10s 456ms/step - loss: 1.6213 - accuracy: 0.3497 - val_loss: 1.3257 - val_accuracy: 0.4375
Epoch 4/20
21/21 [==============================] - 11s 492ms/step - loss: 1.5976 - accuracy: 0.3542 - val_loss: 1.4096 - val_accuracy: 0.4271
Epoch 5/20
21/21 [==============================] - 12s 551ms/step - loss: 1.5121 - accuracy: 0.3705 - val_loss: 1.2429 - val_accuracy: 0.4479
Epoch 6/20
21/21 [==============================] - 11s 494ms/step - loss: 1.4439 - accuracy: 0.4122 - val_loss: 1.3415 - val_accuracy: 0.4635
Epoch 7/20
21/21 [==============================] - 11s 496ms/step - loss: 1.3704 - accuracy: 0.4524 - val_loss: 1.2892 - val_accuracy: 0.4635

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas

In [17]:
import pandas as pd

1/1 [==============================] - 0s 30ms/step
77


In [18]:
from tensorflow.keras.models import load_model
model2.save('skin_disease_classifier.h5')
tf.saved_model.save(model2,'/Users/abhilashhathwar/Desktop/Desk/ERSA/backend')

INFO:tensorflow:Assets written to: /Users/abhilashhathwar/Desktop/Desk/ERSA/backend/assets


INFO:tensorflow:Assets written to: /Users/abhilashhathwar/Desktop/Desk/ERSA/backend/assets


In [19]:
# model_json = model2.to_json()
# with open("model4.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5


In [20]:
# #from keras import model_from_json 

# # opening and store file in a variable

# json_file = open('model4.json','r')
# loaded_model_json = json_file.read()
# json_file.close()

# # use Keras model_from_json to make a loaded model

# loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# # load weights into new model

# loaded_model.load_weights("softstoryclassifier.h5")
# print("Loaded Model from disk")

# # compile and evaluate loaded model

# loaded_model.compile(loss=tf.losses.BinaryCrossentropy(),optimizer='adam',metrics=['accuracy'])


model = tf.keras.models.load_model('skin_disease_classifier.h5')
img='/Users/abhilashhathwar/Desktop/Desk/ERSA/backend/train/Melanoma/atypical-nevi-8.jpg'
new_image = cv2.imread(img)
imResized = cv2.resize(new_image,(64,64))
      #img = cv2.imread(, cv2.IMREAD_COLOR)
      # print(imResized)
imResized = imResized/255.0
# print(imResized.shape)
      #res = cv2.resize(img, dsize=(255, 255))
      #imResized = imResized.reshape(256,256,3)
imResized=imResized[None,:,:,:]
out = model.predict(imResized)


1/1 [==============================] - 0s 78ms/step


In [21]:
import pandas as pd

In [22]:
s=pd.Series(out[0]).idxmax()
diseases=['Acne_and_Rosacea','Actinic_Keratosis_Basal_Cell_Carcinoma','Atopic_Dermatitis','Bullous_Disease','Cellulitis_Impetigo',
          'Eczema','Exanthems_and_Drug_Eruptions','Alopecia','STDS','Pigmentation','Lupus','Melanoma','Nail_Fungus','Contact_Dermatitis',
          'Psoriasis','Lyme_Disease','Seborrheic_Keratoses','Systemic_Disease','Ringworm','Urticaria_Hives','Vascular_Tumors','Vasculitis',
          'Warts_Molluscum']
print(diseases[s])

Cellulitis_Impetigo


In [23]:
model=tf.keras.models.load_model('skin_disease_classifier.h5')
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)

#convert
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/mc/0pln0cqn53s2r30bcb9p9zbc0000gn/T/tmp2yabn8w9/assets


INFO:tensorflow:Assets written to: /var/folders/mc/0pln0cqn53s2r30bcb9p9zbc0000gn/T/tmp2yabn8w9/assets
2023-04-06 18:11:06.756844: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-06 18:11:06.756996: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-06 18:11:06.758289: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/mc/0pln0cqn53s2r30bcb9p9zbc0000gn/T/tmp2yabn8w9
2023-04-06 18:11:06.759845: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-06 18:11:06.759855: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/mc/0pln0cqn53s2r30bcb9p9zbc0000gn/T/tmp2yabn8w9
2023-04-06 18:11:06.764888: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-04-06 18:11:06.766706: I tensorflow/cc/saved_model/load

35420424

In [24]:

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/Users/abhilashhathwar/Desktop/Desk/ERSA/backend/') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model_saved.tflite', 'wb') as f:
  f.write(tflite_model)

2023-04-06 18:11:09.776258: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-06 18:11:09.776272: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-06 18:11:09.776365: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /Users/abhilashhathwar/Desktop/Desk/ERSA/backend/
2023-04-06 18:11:09.777593: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-06 18:11:09.777598: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /Users/abhilashhathwar/Desktop/Desk/ERSA/backend/
2023-04-06 18:11:09.780975: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-04-06 18:11:09.954967: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /Users/abhilashhathwar/Desktop/Desk/ERSA/backend/
2023-04-06 18:11:09.964452: I tensorflow/